In [ ]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, plot_confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving example.csv to example (1).csv
Saving predict.csv to predict (1).csv
Saving train.csv to train (1).csv


In [ ]:
train = pd.read_csv('train.csv')
predict = pd.read_csv('predict.csv')
example = pd.read_csv('example.csv')

In [ ]:
train.head(10)

,Vmag,Plx,e_Plx,B-V,SpType,Amag,TargetClass
0,6.901420,24.569658,0.862829,0.737416,G5V,20.449126,1
1,4.334196,2.198947,0.593309,1.153210,G2Ib,13.043324,0
2,7.873019,11.750465,0.616568,0.589166,F5V,17.736492,1
3,8.110014,2.143815,0.848508,1.322107,K2III,15.592469,0
4,8.025193,4.165622,0.899296,1.141646,K5III,16.379680,0
5,2.899130,11.110086,0.701719,1.095357,M0III,14.370388,0
6,7.529711,9.160176,1.131393,0.926519,G8IV,15.938899,1
7,6.083824,3.928857,0.835288,1.761544,M2.5IIIa,14.063334,0
8,3.000914,5.713252,0.757593,0.113611,A0V,14.350210,1
9,8.706528,1.816360,1.409039,0.250017,A0V,14.763104,1


In [ ]:

train['bukva'] = 0
train['first'] = 0
train['second'] = 0
for index, row in train.iterrows():
  train.loc[index, 'bukva'] = re.split(r'\d', train.loc[index, 'SpType'])[0]
  try:
    a = re.findall(r'.*[0-9]([IV]*)', train.loc[index, 'SpType'])[-1]
    if a  not in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']:
      train.loc[index, 'first'] = 'IV'
    else:
      train.loc[index, 'first'] = a

    train.loc[index, 'second'] = re.findall(r'.([0-9])', train.loc[index, 'SpType'])[0]
  except IndexError:
      train.loc[index, 'first'] = 'IV'
      train.loc[index, 'second'] = 4


train

In [ ]:
train = train.drop(['SpType'], axis=1)
train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,bukva,first,second
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,8,4,5
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,8,0,2
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,7,4,5
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,12,2,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,12,2,5
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,0,1,0
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,8,2,8
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,7,3,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,12,2,0


In [ ]:
from sklearn import preprocessing
 

l1 = preprocessing.LabelEncoder()
l1.fit(train['bukva'])
train['bukva'] = l1.transform(train['bukva'])

l2 = preprocessing.LabelEncoder()
l2.fit(train['first'])
train['first'] = l2.transform(train['first'])


train

,Vmag,Plx,e_Plx,B-V,Amag,TargetClass,bukva,first,second
0,6.901420,24.569658,0.862829,0.737416,20.449126,1,8,4,5
1,4.334196,2.198947,0.593309,1.153210,13.043324,0,8,0,2
2,7.873019,11.750465,0.616568,0.589166,17.736492,1,7,4,5
3,8.110014,2.143815,0.848508,1.322107,15.592469,0,12,2,2
4,8.025193,4.165622,0.899296,1.141646,16.379680,0,12,2,5
...,...,...,...,...,...,...,...,...,...
31636,7.561113,0.692340,0.843319,0.252874,12.542120,0,0,1,0
31637,8.078011,3.002784,0.689891,0.971966,15.379710,0,8,2,8
31638,7.065964,7.820498,0.885794,0.435381,16.737442,1,7,3,3
31639,8.685452,2.643033,1.064647,0.988600,14.859882,0,12,2,0


In [ ]:
X = train.drop(['TargetClass'], axis=1)
y = train['TargetClass']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 1/3)

In [ ]:
rfc = RandomForestClassifier(random_state = 42, n_jobs = -1)
rfc.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [ ]:
pred = rfc.predict(X_test)

In [ ]:
roc_auc_score(y_test, pred)

0.9597209651708986

In [ ]:
predict['bukva'] = 0
predict['first'] = 0
predict['second'] = 0
for index, row in predict.iterrows():
  predict.loc[index, 'bukva'] = re.split(r'\d', predict.loc[index, 'SpType'])[0]
  try:
    a = re.findall(r'.*[0-9]([IV]*)', predict.loc[index, 'SpType'])[-1]
    if a  not in ['I', 'II', 'III', 'IV', 'V', 'VI', 'VII']:
      predict.loc[index, 'first'] = 'IV'
    else:
      predict.loc[index, 'first'] = a

    predict.loc[index, 'second'] = re.findall(r'.([0-9])', predict.loc[index, 'SpType'])[0]
  except IndexError:
      predict.loc[index, 'first'] = 'IV'
      predict.loc[index, 'second'] = 4
predict

,Vmag,Plx,e_Plx,B-V,SpType,Amag,bukva,first,second
0,4.039519,13.596504,0.845101,-0.097823,F5V+...,15.774122,F,V,5
1,6.655719,4.010332,1.050017,1.386670,K5III,15.344976,K,III,5
2,6.405060,0.348855,0.961598,0.951503,B0IVpe,12.236103,B,IV,0
3,9.113068,2.954461,1.251422,1.002819,K1III,15.560838,K,III,1
4,6.107686,3.144180,0.475795,0.493345,G8II-IIIvar,12.771981,G,II,8
...,...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,M3III,16.438014,M,III,3
7907,6.255358,3.034436,0.449386,0.966053,K0III,13.898403,K,III,0
7908,8.643598,10.932145,0.947897,0.504383,G0V,18.980729,G,V,0
7909,9.400810,6.279361,1.332124,0.351073,F2V,16.872491,F,V,2


In [ ]:
from sklearn import preprocessing
 

l4 = preprocessing.LabelEncoder()
l4.fit(predict['bukva'])
predict['bukva'] = l4.transform(predict['bukva'])

l5 = preprocessing.LabelEncoder()
l5.fit(predict['first'])
predict['first'] = l5.transform(predict['first'])

predict

,Vmag,Plx,e_Plx,B-V,SpType,Amag,bukva,first,second
0,4.039519,13.596504,0.845101,-0.097823,F5V+...,15.774122,5,4,5
1,6.655719,4.010332,1.050017,1.386670,K5III,15.344976,8,2,5
2,6.405060,0.348855,0.961598,0.951503,B0IVpe,12.236103,2,3,0
3,9.113068,2.954461,1.251422,1.002819,K1III,15.560838,8,2,1
4,6.107686,3.144180,0.475795,0.493345,G8II-IIIvar,12.771981,6,1,8
...,...,...,...,...,...,...,...,...,...
7906,8.255272,2.858885,0.822289,1.349195,M3III,16.438014,10,2,3
7907,6.255358,3.034436,0.449386,0.966053,K0III,13.898403,8,2,0
7908,8.643598,10.932145,0.947897,0.504383,G0V,18.980729,6,4,0
7909,9.400810,6.279361,1.332124,0.351073,F2V,16.872491,5,4,2


In [ ]:
predict = predict.drop(['SpType'], axis=1)
predict

In [ ]:
X = predict
y = train['TargetClass']

In [ ]:
pred = rfc.predict(predict)

In [ ]:
pred = pd.Series(pred)

In [ ]:
pred.to_csv('Result.csv')
pred_file = pd.read_csv('/content/Result.csv')

In [ ]:
pred_file.rename(columns={'0': 'TargetClass','Unnamed: 0':'Number'}, inplace=True)

In [ ]:
predict=predict.join(pred_file)

In [ ]:
pred_final = predict.drop(['Number'], axis='columns')

In [ ]:
pred_final.to_csv('Result_final.csv',index=False)

In [ ]:
final = pd.read_csv('Result_final.csv')
final = final.drop(['bukva', 'first', 'second'], axis = 1)
final

In [ ]:
final = final.reindex(columns=['Vmag', 'Plx', 'e_Plx', 'B-V', 'SpType', 'Amag', 'TargetClass'])
final

In [ ]:
final.to_csv('Result_final.csv',index=False)